1. Menetukan library yang digunakan

In [57]:
import numpy as pd
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

2. Load dataset

In [58]:
df = pd.read_csv('diabetes.csv')

In [59]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [60]:
#memisahkan fitur dan label
#ambil kolom untuk label diskrit (class)
y = (df['Outcome'])

#mengambil kolom sebagai fitur
x = df.iloc[:, 0:-1]

In [61]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [62]:
#scaler = MinMaxScaler(feature_range=(0, 10))
scaler = StandardScaler()
scaler.fit(x_train)

import joblib

joblib.dump(scaler, 'scaler_diabetes.sav')
scaler = joblib.load('scaler_diabetes.sav')

#scaler.data_max_
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

4. membuat data latih

In [63]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# defining parameter range
param_grid = [
    #{'weights':['uniform','distance'], 'n_neighbors':[3,5,7,9,11,13,15], 'metric':['euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean']}
    #{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    #{'C': [1, 10], 'gamma': [0.01, 0.001], 'kernel': ['rbf']},
    #{'n_estimators': [50, 100, 150, 200], 'criterion':['gini', 'entropy'], 'max_depth':[5, 10, 15], 'min_samples_split':[0.1, 1.0, 10], 'min_samples_leaf':[0.1, 0.5, 5]}
    #{'criterion':['gini', 'entropy'], 'max_depth':[5, 10, 15], 'min_samples_split':[0.1, 1, 10, 20], 'min_samples_leaf':[0.1, 0.5, 5, 10]}
    {'n_estimators': [50, 100, 150, 200],'learning_rate': [0.1,0.2,0.3],}
    #{'hidden_layer_sizes':[(10,10,10),(150,100,50)], 'max_iter':[200,300], 'activation': ['relu','tanh','logistic'], 'solver': ['adam']}
 ]

classifier = AdaBoostClassifier()

#tolong perhatikan parameter scoring dan cv
model = GridSearchCV(classifier, param_grid, scoring='accuracy', cv=10, refit = True, verbose = 3)

# fitting the model for grid search
model.fit(x_train, y_train)

# print best parameter after tuning
print('parameter terbaik:',model.best_params_)

# print how our model looks after hyper-parameter tuning
print(model.best_estimator_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV 1/10] END learning_rate=0.1, n_estimators=50;, score=0.704 total time=   0.1s
[CV 2/10] END learning_rate=0.1, n_estimators=50;, score=0.796 total time=   0.1s
[CV 3/10] END learning_rate=0.1, n_estimators=50;, score=0.815 total time=   0.0s
[CV 4/10] END learning_rate=0.1, n_estimators=50;, score=0.778 total time=   0.0s
[CV 5/10] END learning_rate=0.1, n_estimators=50;, score=0.796 total time=   0.0s
[CV 6/10] END learning_rate=0.1, n_estimators=50;, score=0.759 total time=   0.1s
[CV 7/10] END learning_rate=0.1, n_estimators=50;, score=0.704 total time=   0.2s
[CV 8/10] END learning_rate=0.1, n_estimators=50;, score=0.679 total time=   0.2s
[CV 9/10] END learning_rate=0.1, n_estimators=50;, score=0.755 total time=   0.1s
[CV 10/10] END learning_rate=0.1, n_estimators=50;, score=0.830 total time=   0.1s
[CV 1/10] END learning_rate=0.1, n_estimators=100;, score=0.685 total time=   0.3s
[CV 2/10] END learning_rate=0.1, 

In [64]:
predictions = model.predict(x_test)
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       157
           1       0.68      0.54      0.60        74

    accuracy                           0.77       231
   macro avg       0.74      0.71      0.72       231
weighted avg       0.76      0.77      0.76       231



5. save model

In [65]:
import joblib

# save the model to disk
filename = 'diabetes_predict.sav'
joblib.dump(model.best_estimator_, filename)

# load the model from disk
loaded_model = joblib.load(filename)
print(loaded_model)
#result = loaded_model.score(X_test, y_test)
print('Hasil model menggunakan model yang sudah disimpan:',loaded_model.score(x_test, y_test))
print('Hasil model menggunakan model hasil training langsung:',model.score(x_test, y_test))

AdaBoostClassifier(learning_rate=0.1, n_estimators=200)
Hasil model menggunakan model yang sudah disimpan: 0.7705627705627706
Hasil model menggunakan model hasil training langsung: 0.7705627705627706
